In [1]:
# George Miloshevich 2022

import os, sys
import pickle
from pathlib import Path

from functools import partial # allows us to create a function with arguments passed
folder = Path('./xforanalogs/NA24by48/global_logit/yrs500/interT15')  # The name of the folder where the weights have been stored


import logging
from colorama import Fore # support colored output in terminal
from colorama import Style
if __name__ == '__main__':
    logger = logging.getLogger()
    logger.handlers = [logging.StreamHandler(sys.stdout)]
else:
    logger = logging.getLogger(__name__)
logger.level = logging.INFO     

import importlib.util
def module_from_file(module_name, file_path): #The code that imports the file which originated the training with all the instructions
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            return module

from importlib import import_module
#foo = import_module(fold_folder+'/Funs.py', package=None)


#folder = './xforanalogs/NA24by48/Z8/yrs500/interT15fw20.1.20lrs4'
foo = module_from_file("foo", f'{folder}/Funs.py')
import pickle
import random as rd  
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.metrics import log_loss

tff = foo.tff # tensorflow routines 
ut = foo.ut # utilities
ln = foo.ln #Learn2_new.py
print(ln)
print(ut)
print(tff)
import committor_analogue as ca
print(ca)

Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


2022-11-26 15:08:58.039102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


path_to_ERA = '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe'/ERA/
Trying to import basemap
In this environment you cannot import Basemap
Trying to import cartopy
Trying to import basemap
In this environment you cannot import Basemap
Trying to import cartopy
Successfully imported cartopy
Successfully imported cartopy
==Checking GPU==
From xforanalogs/NA24by48/global_logit/yrs500/interT15/Funs.py:46: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
==Checking CUDA==
<module 'PLASIM.Learn2_new' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/Learn2_new.py'>
<module 'ERA.utilities' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/ERA/utilities.py'>
<module 'ERA.TF_Fields' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/ERA/TF_Fields.py'>
path_to_ERA = '/ClimateDynamics/MediumSpace/Clim

2022-11-26 15:09:02.289302: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 15:09:02.318884: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-26 15:09:02.561840: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-26 15:09:02.561887: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (apollo2048g): /proc/driver/nvidia/version does not exist
2022-11-26 15:09:02.561901: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices

<module 'Learn2_new' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/VAE/../PLASIM/Learn2_new.py'>
<module 'utilities' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/ERA/utilities.py'>
<module 'TF_Fields' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/ERA/TF_Fields.py'>
==Checking GPU==
==Checking CUDA==


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<module 'committor_analogue' from '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/VAE/committor_analogue.py'>


In [2]:
ca.tff = tff
ca.ut = ut
ca.ln = ln


import numba as nb
from numba import jit,guvectorize,set_num_threads
@guvectorize([(nb.int64,nb.int64,nb.int64[:],nb.int64[:],nb.int64,nb.int64[:,:],nb.int64[:,:])],
             '(),(),(n),(m),(),(k,o)->(n,m)',nopython=True,target="parallel") # to enable numba parallelism
def TrajOnePoint(day, nn, n_Traj, numsteps, Markov_step, Matr_tr,res): # a day implies the temporal coordinates in days of the input from the 0-th year of the 1000 year long dataset
    """ Compute `n_Traj` trajectories starting from `day` that take `numsteps` steps
    Args:
        day (_int_):            day in the validation set where the Markov chain will start
        nn (_int_):             number of nearest neighbors to look for
        n_Traj (_int array_):         Number of MC samples that start from the same day (number of trajectories).
        numsteps (_int array_):       numbe of steps to take in each trajectory
        Markov_step (_int_):    step in the Markov chain (how many days)
        Matr_tr (_ndarray_):    T matrix inside the training set
        res (_float_):          stores the committor (return), this is how numba vectorization forces the output to be treated
    """
    wrong_index = 0 # This checks that during input or execution we were always working with indecies that exist in the considered matrices and we don't go below or above
    if (day >= Matr_tr.shape[0]) or (day < 0): # We don't allow inputs that are outside of the range of Matr_va. 
        #print("day > Matr_va.shape[0]")
        wrong_index = 1 # manual debugging (unfortunately numba does not capture this)
        print("We don't allow inputs that are outside of the range of Matr_va")
        for l_1 in n_Traj:
            for l_2 in numsteps:
                res[l_1][l_2] = np.nan # we simply  don't have corresponding index
    if nn > Matr_tr.shape[1]:
        wrong_index = 1 # manual debugging: use to monitor if we get out of the Matr_tr allowed set
        print("We don't allow inputs that are outside of the range of Matr_va")
    else:
        #print("day <= Matr_va.shape[0]")
        for i in n_Traj:   
            s = day # We initialize trajectory at the first day
            res[i][0] = s
            #print("output: ", day,app,s, Matr_va.shape)
            if (s >= Matr_tr.shape[0]) or (s < 0):
                wrong_index = 1
            for j in numsteps[1:]: 
                    app = rd.randint(0,nn-1) #analogue selection
                    s = Matr_tr[s][app] + Markov_step         #analog state s is evolved in time
                    res[i][j] = s
                    if (s >= Matr_tr.shape[0]) or (s < 0):
                        wrong_index = 1
                    if nn > Matr_tr.shape[1]:
                        wrong_index = 1 # manual debugging: use to monitor if we get out of the Matr_tr allowed set
        if wrong_index == 1:
            print("Somewhere inside the code there was an input outside of the range of matrices/vectors")
            for i in n_Traj:
                for j in numsteps:
                    res[l_1][l_2] = np.nan # we simply  don't have the corresponding index
        
def DressTrajOnePoint(day, ther,dela, Temp_va, Temp_tr, nn, n_Traj, numsteps, Markov_step, Matr_va, Matr_tr):
    """This function calls TrajOnePoint by selecting the right inputs provided to ca.CommOnePoint()
    """
    n_Traj = np.arange(n_Traj)
    numsteps = np.arange(numsteps) # Note that this overwrites what was prescribed in ca.RunNeighbors(). We want to simulate a day in summer
    print(f'{n_Traj = }')
    return TrajOnePoint(day, nn, n_Traj, numsteps, Markov_step, Matr_tr)

In [3]:
ca.CommOnePoint = DressTrajOnePoint

ln.RunCheckpoints = ca.RunCheckpoints
ln.RunNeighbors = ca.RunNeighbors
ln.RunFolds = ca.RunFolds


#from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.insert(1, '../ERA')
year_permutation = np.load(f'{folder}/year_permutation.npy')
run_vae_kwargs = ut.json2dict(f"{folder}/config.json")
T = ut.extract_nested(run_vae_kwargs, 'T')
if (ut.keys_exists(run_vae_kwargs, 'label_period_start') and ut.keys_exists(run_vae_kwargs, 'label_period_end')):
    label_period_start = ut.extract_nested(run_vae_kwargs, 'label_period_start')
    label_period_end = ut.extract_nested(run_vae_kwargs, 'label_period_end')
    time_start = ut.extract_nested(run_vae_kwargs, 'time_start')
    time_end = ut.extract_nested(run_vae_kwargs, 'time_end')
threshold = np.array([np.load(f'{folder}/threshold.npy')]) #Threshold defining committor. This parameter I don't need, I shall perhaps transform it into epochs for variational autoencoder 
percent = ut.extract_nested(run_vae_kwargs, 'percent')
nfolds = ut.extract_nested(run_vae_kwargs, 'nfolds')
n_days = time_end-time_start-T+1   
logger.info(f"{Style.RESET_ALL}")

extra_day=1
if ut.keys_exists(run_vae_kwargs, 'A_weights'):
    A_weights = ut.extract_nested(run_vae_kwargs, 'A_weights')
    if A_weights is not None:
        extra_day = A_weights[0] # We need to see if the labels were interpolated to see how much the algorithm should jump each summer
if extra_day == 3:
    delay = np.arange(6)
else:
    delay = 3*np.arange(6)

RunFolds_kwargs_default = ln.get_default_params(ca.RunFolds, recursive=True)
RunFolds_kwargs_default = ut.set_values_recursive(
    RunFolds_kwargs_default, {'num_Traj' : 10, 'chain_step' : extra_day,  'threshold' : threshold,
                                'delay' : delay, 'neighbors' : [3,5,10,20,40], 'num_steps' : n_days - (label_period_start-time_start),
                              'T' : T, 'allowselfanalogs' : True, 'input_set' : 'va', 'bulk_set' : 'tr',
                              'start_calendar_day' :(label_period_start-time_start), 'start_day_set' : 'tr'}  )

chain_step = ut.extract_nested(RunFolds_kwargs_default, 'chain_step')  
logger.info(RunFolds_kwargs_default)




#logger.info(f'{sec[10][10].shape = }')

#logger.info(sec[10][10][0,0])

#logger.info(sec[10][10][0,0]%n_days)


{'nfield': 0, 'input_set': 'va', 'bulk_set': 'tr', 'RunCheckpoints_kwargs': {'start_calendar_day': 15, 'start_day_set': 'tr', 'allowselfanalogs': True, 'RunNeighbors_kwargs': {'num_Traj': 10, 'T': 15, 'chain_step': 3, 'neighbors': [3, 5, 10, 20, 40], 'delay': array([0, 1, 2, 3, 4, 5]), 'num_steps': 90}}}


In [4]:
logger.info(f"{Fore.BLUE}") #  indicates we are inside the routine 

with open(f'{folder}/fold_{0}/analogues.pkl', "rb") as open_file:
    analog = pickle.load(open_file)
analogues_tr = list(analog['ind_new_tr'].values())[0] # Here we need to load just random analogs for the compilation below
time_series_tr = np.load(f"{folder}/fold_{0}/time_series_tr.npy")[:,0]
#sec_1 = TrajOnePoint(33, [2,3,5,10,20,50],10, 5, chain_step, analogues_tr) # compiling (maybe we only need this once)
sec = ca.RunFolds(folder,1, threshold, n_days, **RunFolds_kwargs_default)[0]   # We only run 1 fold
logger.info(f"{Style.RESET_ALL}")

time_series_synth = {k: {j:time_series_tr[u] for j, u in v.items()} for k, v in sec.items()}
#logger.info(f'{time_series_synth[10][10].shape = }')
convolve_vec = np.vectorize(partial(np.convolve, **{'mode':'valid'}), signature='(n),(m)->(k)')
A_synth = {k: {j:convolve_vec(u, np.ones(T)/T) for j, u in v.items()} for k, v in time_series_synth.items()}
logger.info(f'{A_synth[10][10].shape = }')
logger.info('Saving the synthetic time series in ')


RunFolds:
    fold = 0, loading from xforanalogs/NA24by48/global_logit/yrs500/interT15/fold_0/analogues.pkl
    RunCheckpoints:
        checkpoint = 10.0
        Matr_va.shape = (5250, 100)
        days.shape = (450,)        
        RunNeighbors:
            num_Traj = 10, N_Steps = 90, chain_step = 3, T = 15, neighbors = [3, 5, 10, 20, 40], delay = array([0, 1, 2, 3, 4, 5]), threshold = array([2.74272833])
            nn = 3
            n_Traj = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])            
            nn = 5
            n_Traj = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])            
            nn = 10
            n_Traj = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])            
            nn = 20
            n_Traj = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])            
            nn = 40
            n_Traj = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])            
            q[nn].shape = (450, 10, 90)
        RunNeighbors: completed in 0.1 s
        checkpoint = 50.0
        Matr_va.shape = (52